# Binary evolutions for the progenitor of J08408

Study of grid of binaries with a NS orbiting around a massive star. The aim of this notebook is to present a clear picture on the parameters which are related to J08408

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.optimize import bisect
import pandas as pd

# some constants
one_third = 1e0/3e0
pi = 3.1415926535897932384626433832795028841971693993751e0
standard_cgrav = 6.67428e-8
Msun = 1.9892e33
Rsun = 6.9598e10
Hubble_time = 13.461701658024014e0  # Gyr
clight = 2.99792458e10
secyer = 3.1558149984e7

# ---------------------------------------------
# Set some global options
np.seterr(all='ignore')

# ---------------------------------------------
# Load data
root = '..'
run_location = f'{root}/data/natal-kicks'
mesa_run_location = f'{root}/data'
config_location = f'{root}/config'

idgrid, Pgrid, agrid, egrid, probgrid = np.loadtxt(f'{run_location}/target_orbits.data', skiprows=9, unpack=True)

# load summary of mesa simulations
summary_of_progenitors = pd.read_csv(f'{mesa_run_location}/summary_of_star_plus_ns.data', delim_whitespace=True, skiprows=3)
additional_info_on_progenitors = pd.read_csv(f'{mesa_run_location}/additional_info_of_star_plus_ns.data', delim_whitespace=True)
detail_progenitors = pd.read_csv(f'{mesa_run_location}/details_of_progenitors_of_j08408.data', delim_whitespace=True)

mpl_style = f'{config_location}/style.mpl'
paper_style = f'{config_location}/paper-style.mpl'

## Parameters at core collapse

In [ ]:
m1precc = 8.3525627823294126E+000
m2precc = 3.2637938645139933E+001
aprecc = 7.3604203503769526E+001
pprecc = 1.1424984631360623E+001
r1precc = 2.4346822488037647E+000
r2precc = 1.8891193348004055E+001
remnantmass = 1.6601196874643882E+000

## Kepler law & some auxiliary functions

In [ ]:
def a_to_P(separation, m1, m2):
    """Kepler law to go from separation to orbital period
    
    Parameters
    ----------
    a: binary separation in [Rsun]
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]

    Returns
    -------
    period: orbital period in [days]
    """
    separation = separation * Rsun  # in cm
    m1 = m1 * Msun
    m2 = m2 * Msun   # in g

    period = 2 * np.pi * np.sqrt(separation**3/(standard_cgrav*(m1+m2)))/(86400.0)
    return period

def P_to_a(period, m1, m2):
    """
    Binary separation from known period

    Parameters
    ----------
    P: binary period in [days]
    M1: mass of primary star in [Msun]
    M2: mass of secondary star in [Msun]

    Returns
    -------
    a: binary separation in [Rsun]
    """
    period = period * 24e0 * 3600e0  # in sec
    m1 = m1 * Msun; m2 = m2 * Msun   # in g
    tmp = standard_cgrav * (m1 + m2) * (period/(2*np.pi)) ** 2
    separation = np.power(tmp, one_third)
    return separation/Rsun

def rlobe(m1, m2, separation):
    """Roche lobe approximation for a star using Eggleton (1983)
    formula

    Parameters
    ----------
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]
    separation: binary separation in [Rsun]

    Returns
    -------
    RL: Roche lobe of primary star in [Rsun]
    """
    one_third = 1e0/3e0
    q = np.power(m1/m2, one_third)
    RL = 0.49e0 * q**2 / (0.6 * q**2 + np.log(1+q))
    RL = separation * RL

    return RL

---

### Basic summary

In [ ]:
codes = list(set(summary_of_progenitors['termination_code_pm']))
for code in codes:
    mask = summary_of_progenitors['termination_code_pm'] == code
    print('{}:'.format(code), 'n-simulations =', len(summary_of_progenitors[mask]))

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(2.7,2.7))
ax.set_xscale('log');

# axis labels
ax.set_xlabel('period [days]')
ax.set_ylabel('eccentricity');

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0, 1);

# MESA runs
end_colors = ['firebrick', 'steelblue'] #, 'firebrick']
codes_dict = {'ce_merge': 'CE merge',
              'high_atm_mdot': 'high $\\dot{M}_{\\rm RLOF, atm}$'}
for k, code in enumerate(codes_dict.keys()):
    mask = summary_of_progenitors['termination_code_pm'] == code
    seps_cc = summary_of_progenitors['porb_pm'][mask]
    eccs_cc = summary_of_progenitors['e_pm'][mask]
    ax.scatter(seps_cc, eccs_cc, s=2, marker='o',
        c=end_colors[k], label='{}'.format(code))

# text on conditions at CC
ax.annotate(
    'Conditions at CC\n{} = {:.2f} {}\n{} = {:.2f} {} {} {:.2f} {}\n{} = {:.2f} {}'.format(
    r'$a$', aprecc, r'R$_\odot$', r'$m_1$', m1precc, r'M$_\odot$', r'$\rightarrow$',
    remnantmass, r'M$_\odot$', r'$m_2$', m2precc, r'M$_\odot$'),
    fontsize=8,
    xy=(0.48,0.30), xycoords='figure fraction');

# legends
f = lambda c, s: plt.plot([],[], ls='none', marker='o', color=c, markersize=s)[0]
handles = [f(color,4) for color in end_colors]
legends = ['{}'.format(codes_dict[event]) for event in codes_dict.keys()]
fig.legend(handles, legends, loc='center', ncol=3, bbox_to_anchor=(0.51,0.91));

## Eccentricity & orbital periods

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(2.7,2.7))
ax.set_xscale('log');

# axis labels
latex = '{\\rm orb}'
ax.set_xlabel(f'$P_{latex}$ [days]')
ax.set_ylabel('eccentricity');

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0, 1);

# colormap settings
bounds = np.arange(0,5,1)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
cm = plt.get_cmap('RdYlBu')

mask = additional_info_on_progenitors['MT_case'] == 'none'
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['min_chi2'][mask]
ax.scatter(x, y, s=1, c=z, cmap=cm, norm=norm, alpha=0.9, marker='o')

mask = additional_info_on_progenitors['MT_case'] != 'none'
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['min_chi2'][mask]
cb = ax.scatter(x, y, s=3, c=z, cmap=cm, norm=norm, alpha=0.9, marker='o');

cbar = fig.colorbar(cb, ax=ax, label='$\\min(\\chi^2)$', extend='max')
cbar.minorticks_off();

In [ ]:
additional_info_on_progenitors.keys()

In [ ]:
detail_progenitors.keys()

## X-ray binary phase timescale

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(2.7,2.7))
ax.set_xscale('log');

# axis labels
latex = '{\\rm orb}'
ax.set_xlabel(f'$P_{latex}$ [days]')
ax.set_ylabel('eccentricity');

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0.49, 0.89);

# colormap settings
# bounds = np.arange(0,6e5,1000)
# norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
# cm = plt.get_cmap('RdYlBu')

mask = additional_info_on_progenitors['t_xrb'] < 100
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['min_chi2'][mask]
ax.scatter(x, y, s=0.1, c='gray', norm=norm, alpha=0.9, marker='o')

mask = additional_info_on_progenitors['t_xrb'] > 100
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['t_xrb'][mask]
cb = ax.scatter(x, y, s=1, c=z, vmin=100, vmax=2000)# , cmap=cm, norm=norm, alpha=0.9, marker='o');

cbar = fig.colorbar(cb, ax=ax, label='$t_{\\rm XRB} [yrs]$', extend='max')
cbar.minorticks_off();